In [1]:
import csv
from flask import Flask, render_template, request
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from afinn import Afinn
import nltk
import pandas as pd
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\whitty\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [6]:
app = Flask(__name__)

# Initialize the sentiment analyzers
sid = SentimentIntensityAnalyzer()
afinn = Afinn()

# Read data from CSV file
data = pd.read_csv("merged_data.csv")

# Debug print to check if data is loaded
print(data.head())

#Expert variety is what we will make a category
varieties = data['expert_variety'].unique().tolist()

print(varieties)
print(set(data['expert_variety']))

   Unnamed: 0         reviews.date  \
0           1  2014-11-25 03:37:34   
1           2  2017-09-20 13:13:29   
2           3  2011-12-22 03:52:36   
3           4  2017-09-13 01:54:31   
4           5  2016-05-18 07:15:02   

                                             name  \
0                                   Pink Moscato    
1            California Roots Moscato White Wine    
2  Martini Rossi174 Asti Spumante Sparkling Wine    
3       California Roots Pinot Grigio White Wine    
4                                Cabernet Shiraz    

                                        reviews.text  \
0  4 750ml bottles for the price of two With way ...   
1                      Delicious and very affordable   
2  Nice fruity and sweet taking sparkling wine. G...   
3  I'm not sure if this is a bad bottle or if it ...   
4  Best of the bunch of all of The wine Cube sku'...   

                                       reviews.title reviews.username  \
0               Yum! Plus, Environmentally

In [11]:
print(data[data['expert_variety'] == "Chenin Blanc"])

    Unnamed: 0 reviews.date                         name  \
35          36   2016-09-05  Mogen David Blackberry Wine   

                                         reviews.text  \
35  This is my all time favorite wine. But i can n...   

                           reviews.title reviews.username  expert_index  \
35  Wish it was in the store more often.           Teresa           383   

   expert_variety                                 expert_description  \
35   Chenin Blanc  Coming from vines planted in 1979 or earlier, ...   

   expert_designation  expert_points  expert_price expert_province  \
35          Old Vines             89            15      Washington   

            expert_region_1  expert_region_2 expert_taster_name  \
35  Walla Walla Valley (WA)  Columbia Valley   Sean P. Sullivan   

   expert_taster_twitter_handle   expert_winery  
35                @wawinereport  L'Ecole No. 41  


: 

In [5]:
# Route for home page
@app.route('/')
def home():
    categories = set(data['expert_variety'])  # Accessing the 'expert_variety' column
    return render_template('index.html', categories=categories)

In [7]:
# Route for analyzing sentiments
@app.route('/analyze', methods=['POST'])
def analyze():
    category = request.form['category']
    selected_data = data[data['expert_variety'] == category]  # Filter based on 'expert_variety'

    if not selected_data:
        return "Category not found", 404

    # Sentiment analysis using TextBlob
    textblob_sentiments = [TextBlob(d['reviews.text']).sentiment for d in selected_data]

    # Sentiment analysis using VADER
    vader_sentiments = [sid.polarity_scores(d['reviews.text']) for d in selected_data]

    # Sentiment analysis using AFINN
    afinn_sentiments = [afinn.score(d['reviews.text']) for d in selected_data]

    return render_template('result.html', category=category, selected_data=selected_data, 
                           textblob_sentiments=textblob_sentiments, vader_sentiments=vader_sentiments,
                           afinn_sentiments=afinn_sentiments)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\whitty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
import csv
from flask import Flask, render_template, request
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from afinn import Afinn
import nltk
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

data = pd.read_csv("merged_data.csv")
data['expert_description'].dropna()

stop_words = set(stopwords.words('english'))

expert_words = []
amateur_words = []

for expert_review in data['expert_description'].dropna():
    words = nltk.word_tokenize(expert_review.lower())
    expert_words.extend([word for word in words if word.isalpha() and word not in stop_words])

for amateur_review in data['reviews.text'].dropna():
    words = nltk.word_tokenize(amateur_review.lower())
    amateur_words.extend([word for word in words if word.isalpha() and word not in stop_words])

    # Count occurrences of words
expert_word_counts = Counter(expert_words)
amateur_word_counts = Counter(amateur_words)

    # Get top ten words for experts and amateurs
top_ten_expert_words = expert_word_counts.most_common(10)
top_ten_amateur_words = amateur_word_counts.most_common(10)

print("Top ten expert words:", top_ten_expert_words)
print("Top ten amateur words:", top_ten_amateur_words)



Top ten expert words: [('flavors', 65), ('wine', 41), ('fruit', 40), ('aromas', 36), ('hints', 32), ('blend', 31), ('like', 28), ('green', 27), ('fine', 27), ('palate', 25)]
Top ten amateur words: [('wine', 74), ('great', 39), ('love', 27), ('good', 24), ('like', 24), ('would', 18), ('wines', 18), ('time', 15), ('taste', 15), ('sweet', 14)]


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\whitty\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\whitty\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\whitty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


: 

In [ ]:
print(data['reviews.text'].dropna().count())